In [1]:
import pandas as pd

In [34]:
ct_sponsors = pd.read_csv('../data_cleaning/processed_data/sponsor_data.csv')
yf_sponsors = pd.read_csv('../data_ingest/raw_data/merged_stock_data.csv')
fda_drugs = pd.read_csv('../data_cleaning/processed_data/filtered_drugs.csv')
hist = pd.read_csv('../data_cleaning/processed_data/price_changes.csv')

In [26]:
len(ct_sponsors.company_ct.unique())

14218

In [35]:
len(yf_sponsors.company_ct.unique())

281

In [9]:
len(yf_sponsors.company_ct.unique())

244

In [36]:
len(fda_drugs.matched_drug_names.unique())

246

In [37]:
fda_drugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2299 entries, 0 to 2298
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ct_id                      2299 non-null   object
 1   fda_id                     2299 non-null   object
 2   fda_company                2299 non-null   object
 3   matched_drug_names         2299 non-null   object
 4   ct_date                    2299 non-null   object
 5   fda_date                   2220 non-null   object
 6   ct_phase                   2172 non-null   object
 7   fda_brand                  2299 non-null   object
 8   fda_generic                2299 non-null   object
 9   fda_active                 2299 non-null   object
 10  ct_name                    2299 non-null   object
 11  ct_otherNames              2299 non-null   object
 12  normalized_fda_drug_names  2299 non-null   object
dtypes: object(13)
memory usage: 233.6+ KB


In [39]:
hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ct_id                      96 non-null     object 
 1   fda_id                     96 non-null     object 
 2   fda_company                96 non-null     object 
 3   matched_drug_names         96 non-null     object 
 4   ct_date                    96 non-null     object 
 5   fda_date                   96 non-null     object 
 6   ct_phase                   96 non-null     object 
 7   fda_brand                  96 non-null     object 
 8   fda_generic                96 non-null     object 
 9   fda_active                 96 non-null     object 
 10  ct_name                    96 non-null     object 
 11  ct_otherNames              96 non-null     object 
 12  normalized_fda_drug_names  96 non-null     object 
 13  ct_avg_price               96 non-null     float64
 

In [44]:
hist.sort_values(by="price_pct_change", ascending=True).head(2)

,ct_id,fda_id,fda_company,matched_drug_names,ct_date,fda_date,ct_phase,fda_brand,fda_generic,fda_active,ct_name,ct_otherNames,normalized_fda_drug_names,ct_avg_price,fda_avg_price,price_pct_change,profit_or_loss
83,NCT04167670,NDA215153,"Phathom Pharmaceuticals, Inc.",amoxicillin,2019-11-19,2022-05-03,['PHASE3'],['VOQUEZNA DUAL PAK'],['VONOPRAZAN FUMARATE AND AMOXICILLIN'],"['AMOXICILLIN', 'VONOPRAZAN FUMARATE']","['Vonoprazan', 'Amoxicillin', 'Clarithromycin'...",[],"amoxicillin, vonoprazan fumarate, voquezna dua...",24.368333,11.666667,-52.123657,-12.701667
82,NCT04167670,NDA215152,"Phathom Pharmaceuticals, Inc.","amoxicillin, clarithromycin",2019-11-19,2022-05-03,['PHASE3'],['VOQUEZNA TRIPLE PAK'],"['VONOPRAZAN FUMARATE, AMOXICILLIN AND CLARITH...","['AMOXICILLIN', 'CLARITHROMYCIN', 'VONOPRAZAN ...","['Vonoprazan', 'Amoxicillin', 'Clarithromycin'...",[],"voquezna triple pak, vonoprazan fumarate, amox...",24.368333,11.666667,-52.123657,-12.701667


In [1]:
import json
import os
import pymongo
import gridfs
import sys

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['clinical_trials']

In [ ]:
file_path = "../data_ingest/raw_data/clinical_trials_raw.json"
raw_json = json.load(open(file_path))

In [ ]:
db.insert_many(raw_json)

In [ ]:
def setup_mongo(json_file, clear_existing=False):
    """Sets up MongoDB and loads the JSON file into GridFS."""
    try:
        client = pymongo.MongoClient(os.getenv("MONGODB_URI", "mongodb://localhost:27017/"))  # Use env var for flexibility
        db_name = os.getenv("MONGO_DB_NAME", "clinical_trials")  # Use env var for database name
        db = client[db_name]
        fs = gridfs.GridFS(db)

        # Optionally clear existing GridFS data
        if clear_existing:
            for file in fs.find():
                fs.delete(file._id)

        # Check if file already exists in GridFS
        filename = os.path.basename(json_file)
        existing_file = fs.find_one({"filename": filename})
        if existing_file:
            print(f"File '{filename}' already exists in GridFS. Skipping upload.")
            return db, fs, existing_file._id

        print("Uploading JSON data into GridFS...")
        with open(json_file, 'rb') as file:  # Open in binary mode for GridFS
            file_id = fs.put(file, filename=filename)

        print("Data upload complete.")
        return db, fs, file_id
    
    except Exception as e:
        print(f"Error setting up MongoDB: {e}")
        sys.exit(1)


if __name__ == "__main__":
    # Traverse to project root dynamically and locate the JSON file
    script_dir = os.path.dirname(os.path.abspath(__file__))
    project_root = os.path.abspath(os.path.join(script_dir, "../"))
    json_file_path = os.path.join(project_root, "data_ingest/raw_data/clinical_trials_raw.json")

    if not os.path.exists(json_file_path):
        print(f"JSON file not found: {json_file_path}")
        sys.exit(1)

    # Initialize MongoDB and load file into GridFS
    print("Initializing MongoDB...")
    db, fs, file_id = setup_mongo(json_file_path, clear_existing=False)
    print(f"MongoDB setup complete. File ID: {file_id}")

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/clinical_trials-completedDate_industry.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71915 entries, 0 to 71914
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   nct_id                         71915 non-null  object
 1   brief_title                    71915 non-null  object
 2   sponsor_name                   71915 non-null  object
 3   phases                         71915 non-null  object
 4   has_results                    71915 non-null  bool  
 5   overall_status                 71915 non-null  object
 6   primary_completion_date        71915 non-null  object
 7   study_first_post_date          71915 non-null  object
 8   url                            71915 non-null  object
 9   clean_primary_completion_date  71915 non-null  object
 10  is_phase4                      71915 non-null  int64 
dtypes: bool(1), int64(1), object(9)
memory usage: 5.6+ MB


In [3]:
df.head()

,nct_id,brief_title,sponsor_name,phases,has_results,overall_status,primary_completion_date,study_first_post_date,url,clean_primary_completion_date,is_phase4
0,NCT04128579,Study of EQ001 (Itolizumab) in Systemic Lupus ...,Equillium,['PHASE1'],False,COMPLETED,2023-11-16,2019-10-16,https://clinicaltrials.gov/study/NCT04128579,2023-11-16,0
1,NCT02312479,Safety and Performance Study of the Nyxoah SAT...,Nyxoah S.A.,['NA'],False,TERMINATED,2015-11,2014-12-09,https://clinicaltrials.gov/study/NCT02312479,2015-11-01,0
2,NCT03235479,Safety and Efficacy Study in Adult Subjects Wi...,Pfizer,['PHASE3'],True,COMPLETED,2018-01-21,2017-08-01,https://clinicaltrials.gov/study/NCT03235479,2018-01-21,0
3,NCT02840279,A Multiple Ascending Dose Study of BPN14770 in...,Tetra Discovery Partners,['PHASE1'],False,COMPLETED,2016-11,2016-07-21,https://clinicaltrials.gov/study/NCT02840279,2016-11-01,0
4,NCT03284879,Post-Marketing Surveillance Study of OTEZLA,Amgen,Unknown,False,COMPLETED,2021-10-31,2017-09-15,https://clinicaltrials.gov/study/NCT03284879,2021-10-31,0


In [6]:
# Strip whitespace and convert all dates, forcing "YYYY-MM" format into "YYYY-MM-01"
df['clean_primary_completion_date'] = (
    df['primary_completion_date']
    .str.strip()  # Remove any surrounding whitespace
    .apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce') 
           if len(x) > 7 else pd.to_datetime(x + '-01', format='%Y-%m-%d'))
)

In [10]:
df['is_phase4'] = df['phases'].apply(lambda x: 1 if x.strip() == "['PHASE4']" else 0)

In [15]:
filtered_df = df[df['clean_primary_completion_date'] >= pd.Timestamp.now() - pd.DateOffset(years=10)]

In [16]:
filtered_df.is_phase4.value_counts()

is_phase4
0    69260
1     2566
Name: count, dtype: int64

In [17]:
len(filtered_df['sponsor_name'].unique())

11994

In [19]:
filtered_df['sponsor_name'].value_counts().head(50)

sponsor_name
Novartis Pharmaceuticals                            1412
Pfizer                                              1361
AstraZeneca                                         1355
GlaxoSmithKline                                     1013
Hoffmann-La Roche                                    948
Bayer                                                893
Eli Lilly and Company                                883
Bristol-Myers Squibb                                 828
Boehringer Ingelheim                                 792
AbbVie                                               738
Merck Sharp & Dohme LLC                              724
Janssen Research & Development, LLC                  671
Takeda                                               607
Novo Nordisk A/S                                     567
Sanofi                                               491
Jiangsu HengRui Medicine Co., Ltd.                   454
Amgen                                                453
Gilead Sciences   